Imports

In [1]:
import cv2
import pandas as pd
import numpy as np
import mediapipe as mp
import os

In [2]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
include_50=pd.read_csv(r'D:/final year project/Train_Test_Split/train_include50.csv')
dataset_path_new = r"D:/final year project/dataset_include50/"
key_points_path=r"D:/final year project/sign-language-recognition/key_points/"
print(include_50)

    Category     Word         Video                       FilePath
0    Animals   4.Bird  MVI_2987.MOV   Animals/4. Bird/MVI_2987.MOV
1    Animals   4.Bird  MVI_3094.MOV   Animals/4. Bird/MVI_3094.MOV
2    Animals   4.Bird  MVI_3095.MOV   Animals/4. Bird/MVI_3095.MOV
3    Animals   4.Bird  MVI_8567.MP4   Animals/4. Bird/MVI_8567.MP4
4    Animals   4.Bird  MVI_3040.MOV   Animals/4. Bird/MVI_3040.MOV
..       ...      ...           ...                            ...
761  Society  2.Death  MVI_8663.MP4  Society/2. Death/MVI_8663.MP4
762  Society  2.Death  MVI_8930.MP4  Society/2. Death/MVI_8930.MP4
763  Society  2.Death  MVI_8664.MP4  Society/2. Death/MVI_8664.MP4
764  Society  2.Death  MVI_4802.MOV  Society/2. Death/MVI_4802.MOV
765  Society  2.Death  MVI_4257.MOV  Society/2. Death/MVI_4257.MOV

[766 rows x 4 columns]


In [11]:
for Category, Word,Video,FilePath in include_50.values:
    file_path=os.path.join(dataset_path_new,FilePath)#remove the filename and extension for folder creation
    print(file_path)
    cap=cv2.VideoCapture(file_path)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()


            # Make detections
            if ret:
                image, results = mediapipe_detection(frame, holistic)                
                # Draw landmarks
                draw_styled_landmarks(image, results)

                # Show to screen
                # cv2.imshow('OpenCV Feed', image)
                extract_keypoints_and_save(results,FilePath,key_points_path)
            else:
                break

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()


D:/final year project/dataset_include50/Animals/4. Bird/MVI_2987.MOV
D:/final year project/dataset_include50/Animals/4. Bird/MVI_3094.MOV
D:/final year project/dataset_include50/Animals/4. Bird/MVI_3095.MOV
D:/final year project/dataset_include50/Animals/4. Bird/MVI_8567.MP4
D:/final year project/dataset_include50/Animals/4. Bird/MVI_3040.MOV
D:/final year project/dataset_include50/Animals/4. Bird/MVI_3096.MOV
D:/final year project/dataset_include50/Animals/4. Bird/MVI_3037.MOV
D:/final year project/dataset_include50/Animals/4. Bird/MVI_3013.MOV
D:/final year project/dataset_include50/Animals/4. Bird/MVI_3012.MOV
D:/final year project/dataset_include50/Animals/4. Bird/MVI_3038.MOV
D:/final year project/dataset_include50/Animals/4. Bird/Extra/MVI_3045.MOV
D:/final year project/dataset_include50/Animals/4. Bird/Extra/MVI_3044.MOV
D:/final year project/dataset_include50/Animals/4. Bird/Extra/MVI_3043.MOV
D:/final year project/dataset_include50/Animals/4. Bird/Extra/MVI_3042.MOV
D:/final y

In [9]:
def extract_keypoints_and_save(results,file_path,key_points):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    result=np.concatenate([pose, face, lh, rh])
    word_folder=os.path.join(key_points,file_path)[:-13]#remove the filename and extension for folder creation
    #before D:/final year project/dataset/Animals/4. Bird/MVI_2987.MOV
    #after D:/final year project/dataset/Animals/4. Bird
    #if folder doesn't exisit, create it
    if not os.path.exists(word_folder):
        print("creating folder:",word_folder)
        os.makedirs(word_folder)
    npy_path=os.path.join(key_points,file_path[:-4]+'.npy')
    np.save(npy_path,result)

Test Data save as key points

In [14]:
key_points_path_test=r"D:/final year project/sign-language-recognition/key_points_test/"
include_50_test=pd.read_csv(r'D:/final year project/Train_Test_Split/test_include50.csv')
dataset_path_new_test = r"D:/final year project/dataset_include50_test/"


In [17]:
for Category, Word,Video,FilePath in include_50_test.values:
    file_path=os.path.join(dataset_path_new_test,FilePath)#remove the filename and extension for folder creation
    print(file_path)
    cap=cv2.VideoCapture(file_path)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()


            # Make detections
            if ret:
                image, results = mediapipe_detection(frame, holistic)                
                # Draw landmarks
                draw_styled_landmarks(image, results)

                # Show to screen
                # cv2.imshow('OpenCV Feed', image)
                extract_keypoints_and_save(results,FilePath,key_points_path_test)
            else:
                break

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()


D:/final year project/dataset_include50_test/Animals/4. Bird/MVI_2988.MOV
creating folder: D:/final year project/sign-language-recognition/key_points_test/Animals/4. Bird
D:/final year project/dataset_include50_test/Colours/54. Black/MVI_4920.MOV
creating folder: D:/final year project/sign-language-recognition/key_points_test/Colours/54. Black
D:/final year project/dataset_include50_test/Colours/54. Black/MVI_3740.MOV
D:/final year project/dataset_include50_test/Colours/54. Black/MVI_3742.MOV
D:/final year project/dataset_include50_test/Colours/54. Black/MVI_5207.MOV
D:/final year project/dataset_include50_test/Colours/54. Black/MVI_4041.MOV
D:/final year project/dataset_include50_test/Means_of_Transportation/11. Car/MVI_3145.MOV
creating folder: D:/final year project/sign-language-recognition/key_points_test/Means_of_Transportation/11. Car
D:/final year project/dataset_include50_test/Means_of_Transportation/11. Car/MVI_3174.MOV
D:/final year project/dataset_include50_test/Means_of_Tra